# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

# Loop through columns and display their data types
#for col, dtype in zip(df.columns, df.dtypes):
#    print(f"{col}: {dtype}")

df.head()


In [ ]:
# CLEANING OF LABELS AND VALUES

# Create a new DataFrame by copying the original df
df_clean = df.copy()

# Clean all column names: replace spaces with underscores and convert to lowercase
df_clean.columns = df_clean.columns.str.replace(' ', '_').str.lower()

# Deleting unwanted columns
del df_clean["unnamed:_0"]

# Round values in 'customer_lifetime_value' column to two decimal places
df_clean['customer_lifetime_value'] = df_clean['customer_lifetime_value'].round(2)

### Date conversion
# Convert to datetime using flexible parsing (will handle M/D/YY automatically)
df_clean['effective_to_date'] = pd.to_datetime(df_clean['effective_to_date'], format='%m/%d/%y', errors='coerce')

# Explicitly define the format, handling both two-digit and four-digit years
df_clean['effective_to_date'] = pd.to_datetime(df_clean['effective_to_date'], format='%m/%d/%Y', errors='coerce')

# Find rows where the date conversion failed (NaT)
invalid_dates = df_clean[df_clean['effective_to_date'].isna()]
### Date conversion

### Replacing 0 values of column 'income' with median values
# Calculate the median of non-zero income values
income_median = df_clean.loc[df_clean['income'] != 0, 'income'].median()

# Replace 0 values with the calculated median
df_clean['income'] = df_clean['income'].replace(0, income_median)
### Replacing 0 values of column 'income' with median values

# Convert the 'number_of_open_complaints' column to integer type
df_clean['number_of_open_complaints'] = df_clean['number_of_open_complaints'].astype('Int64')

# Round values in 'total_claim_amount' column to two decimal places
df_clean['total_claim_amount'] = df_clean['total_claim_amount'].round(2)

# Replace 'Medsize' with 'Medium' in the 'vehicle_size' column
df_clean['vehicle_size'] = df_clean['vehicle_size'].replace('Medsize', 'Medium')

# View the first few rows of the cleaned DataFrame
display(df_clean.head())



In [80]:
# Proofing cell for cleaning
# Print unique values in the 'state' column directly
# print(df_clean['vehicle_type'].unique())

# Create a new DataFrame that only includes customers who:
#    - have a low total_claim_amount (e.g., below $1,000),
#    - have a response "Yes" to the last marketing campaign.


[nan 'A']


In [ ]:
# Filter customers with total_claim_amount below 1000 and response == "Yes"
df_low_claim_yes = df_clean.loc[(df['total_claim_amount'] < 1000) & (df['response'] == 'Yes')]

# Preview the result
df_low_claim_yes[['total_claim_amount', 'response']].head(30)


2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [85]:
# Filter customers who responded "Yes"
df_response_yes = df_clean[df_clean['response'] == 'Yes']

# Group by policy_type and gender, then calculate averages
grouped_means = df_response_yes.groupby(['policy_type', 'gender'])[['monthly_premium_auto', 'customer_lifetime_value', 'total_claim_amount']].mean().reset_index()

# Step 3: Display the results
print("Average Values for Customers Who Responded 'Yes':")
print(grouped_means)

# Optional: Sort by customer_lifetime_value to see high-value segments
sorted_by_value = grouped_means.sort_values(by='customer_lifetime_value', ascending=False)
print("\nSorted by Customer Lifetime Value:")
print(sorted_by_value)

Average Values for Customers Who Responded 'Yes':
      policy_type gender  monthly_premium_auto  customer_lifetime_value  total_claim_amount
0  Corporate Auto      F             94.301775              7712.628521          433.738462
1  Corporate Auto      M             92.188312              7944.465974          408.582468
2   Personal Auto      F             98.998148              8339.791630          452.966037
3   Personal Auto      M             91.085821              7448.383265          457.010149
4    Special Auto      F             92.314286              7691.584571          453.279714
5    Special Auto      M             86.343750              8247.089375          429.528125

Sorted by Customer Lifetime Value:
      policy_type gender  monthly_premium_auto  customer_lifetime_value  total_claim_amount
2   Personal Auto      F             98.998148              8339.791630          452.966037
5    Special Auto      M             86.343750              8247.089375          429.5

Risk (Total Claim Amount)
Lowest Claim Segment:
Corporate Auto – Male customers have the lowest average total claim amount at around 408.6, making them the lowest-risk group to insure.
Special Auto – Male also presents a favorable risk profile with a moderate average claim of approximately 429.5, while still maintaining high customer lifetime value.

Profitability (Customer Lifetime Value)
Top Segment:
Personal Auto – Female customers have the highest average customer lifetime value at approximately 8,340, indicating strong long-term profitability.
This is followed closely by Special Auto – Male and Corporate Auto – Male, both offering high lifetime values above 7,900.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [86]:
# Count number of customers per state
customers_by_state = df_clean['state'].value_counts()

# Filter states with more than 500 customers
states_over_500 = customers_by_state[customers_by_state > 500]

# Display result
print(states_over_500)


state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [87]:
# Group by education and gender, then calculate max, min, and median of customer_lifetime_value
education_gender_clev = df_clean.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median'])

# Display the result
print(education_gender_clev)


                                  max      min    median
education            gender                             
Bachelor             F       73225.96  1904.00  5640.505
                     M       67907.27  1898.01  5548.030
College              F       61850.19  1898.68  5623.610
                     M       61134.68  1918.12  6005.850
Doctor               F       44856.11  2395.57  5332.460
                     M       32677.34  2267.60  5577.670
High School or Below F       55277.45  2144.92  6039.555
                     M       83325.38  1940.98  6286.735
Master               F       51016.07  2417.78  5729.860
                     M       50568.26  2272.31  5579.100


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here